In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import requests
plt.rcParams['figure.figsize'] = (12,6)

In [3]:
def fetch_btc_history(start='2015-01-01', end=None, csv_fallback='data/btc.csv'):
    end = end or datetime.utcnow().strftime('%Y-%m-%d')
    try:
        import yfinance as yf
        df = yf.download('BTC-USD', start=start, end=end, progress=False)
        if df.empty:
            raise RuntimeError('yfinance returned empty dataframe')
        df = df[['Open','High','Low','Close','Volume']].copy()
        df.index = pd.to_datetime(df.index)
        print(f"Downloaded {len(df)} rows from yfinance, range {df.index[0].date()} to {df.index[-1].date()}")
        return df
    except Exception as e:
        print('yfinance fetch failed:', e)
        if os.path.exists(csv_fallback):
            print('Falling back to local CSV:', csv_fallback)
            df = pd.read_csv(csv_fallback, parse_dates=['Date'], index_col='Date')
            for c in ['Open','High','Low','Close','Volume']:
                if c not in df.columns:
                    raise RuntimeError(f'Missing column {c} in fallback CSV')
            return df
        else:
            raise RuntimeError('No data available. Place a CSV at data/btc.csv or enable internet.') from e

# Fetch
df = fetch_btc_history(start='2018-01-01')
df.head()

/var/folders/nq/d8rg086n21lcvj8v0tdfdzbh0000gn/T/ipykernel_82816/4009790293.py:2: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end = end or datetime.utcnow().strftime('%Y-%m-%d')
/var/folders/nq/d8rg086n21lcvj8v0tdfdzbh0000gn/T/ipykernel_82816/4009790293.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('BTC-USD', start=start, end=end, progress=False)


Downloaded 2887 rows from yfinance, range 2018-01-01 to 2025-11-26


Price,Open,High,Low,Close,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,
2018-01-01,14112.200195,14112.200195,13154.700195,13657.200195,10291200000
2018-01-02,13625.000000,15444.599609,13163.599609,14982.099609,16846600192
2018-01-03,14978.200195,15572.799805,14844.500000,15201.000000,16871900160
2018-01-04,15270.700195,15739.700195,14522.200195,15599.200195,21783199744
2018-01-05,15477.200195,17705.199219,15202.799805,17429.500000,23840899072


In [4]:
df.tail()

Price,Open,High,Low,Close,Volume
Ticker,BTC-USD,BTC-USD,BTC-USD,BTC-USD,BTC-USD
Date,,,,,
2025-11-22,85098.562500,85503.007812,83490.898438,84648.359375,40793099246
2025-11-23,84648.609375,88038.468750,84641.773438,86805.007812,58083435576
2025-11-24,86798.773438,89206.335938,85272.195312,88270.562500,74433896110
2025-11-25,88269.960938,88457.335938,86131.429688,87341.890625,64837343545
2025-11-26,87345.585938,90581.156250,86316.898438,90518.367188,66496301869


In [5]:
def add_indicators(df, short_window=50, long_window=200, atr_window=14, rsi_window=14):
    df = df.copy()
    df['SMA_short'] = df['Close'].rolling(short_window).mean()
    df['SMA_long'] = df['Close'].rolling(long_window).mean()
    high_low = df['High'] - df['Low']
    high_close = (df['High'] - df['Close'].shift()).abs()
    low_close = (df['Low'] - df['Close'].shift()).abs()
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR'] = tr.rolling(atr_window).mean()
    delta = df['Close'].diff()
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)
    ma_up = up.rolling(rsi_window).mean()
    ma_down = down.rolling(rsi_window).mean()
    rs = ma_up / (ma_down + 1e-9)
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

# Add indicators
df = add_indicators(df)
df[['Close','SMA_short','SMA_long','ATR','RSI']].tail()

Price,Close,SMA_short,SMA_long,ATR,RSI
Ticker,BTC-USD,,,,
Date,,,,,
2025-11-22,84648.359375,106936.383125,110261.636680,4474.064732,19.675843
2025-11-23,86805.007812,106223.974687,110210.500117,4434.579241,18.892325
2025-11-24,88270.562500,105519.116406,110135.645625,4557.443080,19.420116
2025-11-25,87341.890625,104770.903594,110057.500820,4368.512835,20.914148
2025-11-26,90518.367188,104152.243281,109986.611016,4354.487165,30.621006


In [1]:
def generate_signals(df, use_rsi_filter=False, rsi_buy_thresh=30, rsi_sell_thresh=70):
    df = df.copy()
    df['signal'] = 0
    df['signal'] = np.where(df['SMA_short'] > df['SMA_long'], 1, 0)
    df['signal_prev'] = df['signal'].shift(1).fillna(0).astype(int)
    df['position'] = df['signal'] - df['signal_prev']
    if use_rsi_filter:
        buy_mask = (df['position'] == 1) & (df['RSI'] < rsi_buy_thresh)
        sell_mask = (df['position'] == -1) & (df['RSI'] > rsi_sell_thresh)
        df['position'] = 0
        df.loc[buy_mask, 'position'] = 1
        df.loc[sell_mask, 'position'] = -1
    return df

# Generate
df = generate_signals(df)
df[['Close','SMA_short','SMA_long','position']].tail(8)

NameError: name 'df' is not defined